# Amazon Bedrock

導入庫

In [ ]:
# 導入庫
import boto3
# 建立 AWS 客戶端
bedrock_client = boto3.client(
    service_name='bedrock-agent-runtime'
)

session = boto3.session.Session()
region = session.region_name

調用 API 的函數 `retrieve_and_generate`

In [ ]:
def retrieveAndGenerate(
    input_text,
    sourceType,
    model_id,
    region,
    document_s3_uri=None,
    data=None,
    identifier=None
):
    # URI 
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
    # 假如是 S3
    if sourceType == "S3":
        # 透過客戶端調用 retrieve_and_generate 函數
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri  
                            }
                        }
                    ]
                }
            }
        )
    else:
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "byteContent": {
                                "identifier": identifier,
                                "contentType": "text/plain",
                                "data": data  
                            }
                        }
                    ]
                }
            }
        )

跟文件對話

In [ ]:
# 模型 ID
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# 替換自己的 S3 URI
document_uri = "s3://data-dump-2024/FAQ_Zappos.com.pdf"

In [ ]:
def my_chatbot(question):
    response = retrieveAndGenerate(
        input_text=question,
        sourceType="S3",
        model_id=model_id,
        region=region,
        document_s3_uri=document_uri,
    )
    return response

提問

In [ ]:
my_question = "你們的退費政策是什麼？"
response = my_chatbot(question=my_question)
print(response['output']['text'])

提問

In [ ]:
my_question =  "我需要多長時間才能收到退款？"
response = my_chatbot(question=my_question)
print(response['output']['text'])

查詢版本

In [ ]:
boto3.__version__

# 改為本地文件

In [ ]:
%pip install PyPDF2

In [19]:
import boto3
from PyPDF2 import PdfReader
import base64
import os
from dotenv import load_dotenv

# 環境變數
load_dotenv()

bedrock_client = boto3.client(service_name='bedrock-agent-runtime')

session = boto3.session.Session()
region = session.region_name

def retrieveAndGenerate(
    input_text,
    sourceType,
    model_id,
    region,
    document_content=None,
    identifier=None
):
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'

    if sourceType == "S3":
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_content  
                            }
                        }
                    ]
                }
            }
        )
    else:
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "byteContent": {
                                "identifier": identifier,
                                "contentType": "application/pdf",
                                "data": document_content  
                            }
                        }
                    ]
                }
            }
        )

def read_pdf_as_base64(file_path, max_pages=5):
    pdf_text = ""
    with open(file_path, "rb") as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        for page in range(min(max_pages, len(pdf_reader.pages))):
            pdf_text += pdf_reader.pages[page].extract_text()
        return base64.b64encode(pdf_text.encode('utf-8')).decode('utf-8')

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
document_path = "HuggingFace物件偵測應用.pdf"
pdf_content_base64 = read_pdf_as_base64(document_path)
print(pdf_content_base64)

def my_chatbot(question):
    response = retrieveAndGenerate(
        input_text=question,
        sourceType="BYTE_CONTENT",
        model_id=model_id,
        region=region,
        document_content=pdf_content_base64,
        identifier=document_path
    )
    return response

# 提問一
my_question = "主要技術重點為何？"
response = my_chatbot(question=my_question)
print(response['output']['text'], '\n\n')

# 提問二
my_question =  "最終成品為何？"
response = my_chatbot(question=my_question)
print(response['output']['text'])


T25lQUkg4r2C5Lu2ICgvcy91c2VyLWd1aWRlKQpBSSBNYWtlciDmoYjkvovmlZnlrbggIC0gSHVnZ2luZyBGYWNlIOeJqeS7tuWBtea4rOaHiQrivaQKQUkgTWFrZXIg5qGI5L6L5pWZ5a24ICAtIEh1Z2dpbmcgRmFjZSDnianku7blgbXmuKzmh4nivaQKMC4g5YmN4r6UCjEuIOa6luWCmeizh+aWmembhgoxLjEg6LOH5paZ5qC85byP6Kqq5piOCjEuMiDlu7rnq4vlhLLlrZjpq5QKMS4zIOS4i+i8ieizh+aWmembhgoyLiDoqJPnt7Tnianku7blgbXmuKzku7vli5nmqKHlnosKMi4xIOW7uueri+iok+e3tOS7u+WLmQoyLjIg5ZWf5YuV6KiT57e05Lu75YuZCjIuMyDmqqLoppboqJPnt7TntZDmnpwKMi40IOiou+WGiuaooeWeiwozLiDlu7rnq4vmjqjoq5bmnI3li5kKMy4xIOW7uueri+aOqOirluS7u+WLmQozLjIg6YCy4r6P5o6o6KuWCjAuIOWJjeK+lApIdWdnaW5nIEZhY2UgKGh0dHBzOi8vaHVnZ2luZ2ZhY2UuY28vKSDmmK/lhazplovnmoQgIEFJIGNvbW11bml0eSDvvIzivYDmj7TkuLvmtYHnmoTmt7Hluqblrbjnv5LmoYbmnrbvvIzkuKbmj5AK5L6b4ryk6YeP55qE6aCQ6KiT57e05qih5Z6L44CB6LOH5paZ6ZuG5Y+K6ZaL55m84ryv5YW377yM5YyF5ous4r6D54S26Kqe4r6U6JmV55CG44CB6Kqe4r6z6L6o6K2Y5Y+K5b2x5YOP6L6o6K2Y562J5aSa56iuICBBSQrmh4nivaTjgIIK6YCP6YGOIE9uZUFJIOaPkOS+m+eahCAgwqBodWdnaW5nZmFjZcKgIOWuueWZqOaYoOWDj+aqlO+8jOWSjCAgQUkgTWFrZXIg5Z+6